In [51]:
# 1. Install dependencies
!pip install --upgrade ultralytics kagglehub scikit-learn pyyaml

In [52]:
import os
import shutil
from pathlib import Path
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import yaml
from kagglehub import dataset_download

In [77]:
raw_path = dataset_download("sshikamaru/car-object-detection")
print("Raw data at:", raw_path)
raw_dir = Path(raw_path) / "data"
raw_dir

Raw data at: /home/user/.cache/kagglehub/datasets/sshikamaru/car-object-detection/versions/2


PosixPath('/home/user/.cache/kagglehub/datasets/sshikamaru/car-object-detection/versions/2/data')

In [76]:
csv_path = raw_dir / "train_solution_bounding_boxes (1).csv"
df = pd.read_csv(csv_path)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/user/.cache/kagglehub/datasets/yusufberksardoan/traffic-detection-project/versions/1/data/train_solution_bounding_boxes (1).csv'

In [ ]:
img_dir = raw_dir / "training_images"
all_imgs = sorted(img_dir.glob("*.jpg")) + sorted(img_dir.glob("*.png"))

train_imgs, val_imgs = train_test_split(all_imgs, test_size=0.2, random_state=42)

In [ ]:
yolo_root = Path("car_dataset")
for sub in ["images/train", "images/val", "labels/train", "labels/val"]:
    (yolo_root / sub).mkdir(parents=True, exist_ok=True)

In [ ]:
def write_yolo_label(img_path, rows, out_label_path):
    # open image to get width/height
    w, h = Image.open(img_path).size
    lines = []
    for _, r in rows.iterrows():
        xmin, ymin, xmax, ymax = r[['xmin','ymin','xmax','ymax']]
        # convert to YOLO format
        x_center = ((xmin + xmax) / 2) / w
        y_center = ((ymin + ymax) / 2) / h
        width     = (xmax - xmin) / w
        height    = (ymax - ymin) / h
        # class 0 = car
        lines.append(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
    out_label_path.write_text("\n".join(lines))

In [ ]:
def process_split(img_list, split):
    for img_path in img_list:
        # copy image
        dest_img = yolo_root / f"images/{split}" / img_path.name
        shutil.copy(img_path, dest_img)
        # select all boxes for this image
        rows = df[df['image'] == img_path.name]
        # write .txt (even if no rows, creates empty file)
        dest_lbl = yolo_root / f"labels/{split}" / img_path.with_suffix('.txt').name
        write_yolo_label(img_path, rows, dest_lbl)

process_split(train_imgs, "train")
process_split(val_imgs,   "val")

In [ ]:
data = {
    'path': str(yolo_root.resolve()),
    'train': 'images/train',
    'val':   'images/val',
    'nc':    1,
    'names': ['car']
}
with open('data.yaml', 'w') as f:
    yaml.safe_dump(data, f)
print("data.yaml:")
print(yaml.dump(data, sort_keys=False))

data.yaml:
path: /home/user/Cortana/machine-learning/car_dataset
train: images/train
val: images/val
nc: 1
names:
- car



In [ ]:
from ultralytics import YOLO
model = YOLO('yolov12n.pt')  # or 'yolov8n.pt' if you don't have v12
model.train(
    data='data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    lr0=0.001,
    patience=10,
    augment=True,
    multi_scale=True,
    name='car-detection-exp'
)

Ultralytics 8.3.118 🚀 Python-3.12.2 torch-2.7.0+cu126 CUDA:0 (NVIDIA RTX A6000, 48566MiB)
engine/trainer: task=detect, mode=train, model=yolov12n.pt, data=data.yaml, epochs=100, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=car-detection-exp, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width

train: Scanning /home/user/Cortana/machine-learning/car_dataset/labels/train.cache... 800 images, 523 backgrounds, 0 corrupt: 100%|██████████| 800/800 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 815.9±533.0 MB/s, size: 105.6 KB)


val: Scanning /home/user/Cortana/machine-learning/car_dataset/labels/val.cache... 201 images, 123 backgrounds, 0 corrupt: 100%|██████████| 201/201 [00:00<?, ?it/s]


Plotting labels to runs/detect/car-detection-exp/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 113 weight(decay=0.0), 120 weight(decay=0.0005), 119 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to runs/detect/car-detection-exp
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        201        117      0.972      0.983       0.99      0.664
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 49, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



59 epochs completed in 0.144 hours.
Optimizer stripped from runs/detect/car-detection-exp/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/car-detection-exp/weights/best.pt, 5.5MB

Validating runs/detect/car-detection-exp/weights/best.pt...
Ultralytics 8.3.118 🚀 Python-3.12.2 torch-2.7.0+cu126 CUDA:0 (NVIDIA RTX A6000, 48566MiB)
YOLOv12n summary (fused): 159 layers, 2,556,923 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  4.49it/s]


                   all        201        117      0.971      0.974      0.989      0.707
Speed: 0.1ms preprocess, 5.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/car-detection-exp


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x76b76419c800>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 